Tarquin Bennett

# Question 1

In [22]:
from nltk.corpus import wordnet as wn

words = ["even",
"express",
"fair",
"fast",
"fine"]

highFreqSenses=[]

for word in words:
    highFreqSenses.append(wn.synsets(word))


for synList in highFreqSenses:
    for syn in synList:
        print("High Frequency Senses: ",syn)
        print("Definition: ",wn.synset(syn.name()).definition())
        print("Examples: ",wn.synset(syn.name()).examples())
        print()

High Frequency Senses:  Synset('evening.n.01')
Definition:  the latter part of the day (the period of decreasing daylight from late afternoon until nightfall)
Examples:  ['he enjoyed the evening light across the lake']

High Frequency Senses:  Synset('flush.v.04')
Definition:  make level or straight
Examples:  ['level the ground']

High Frequency Senses:  Synset('even.v.02')
Definition:  become even or more even
Examples:  ['even out the surface']

High Frequency Senses:  Synset('even.v.03')
Definition:  make even or more even
Examples:  []

High Frequency Senses:  Synset('even.a.01')
Definition:  divisible by two
Examples:  []

High Frequency Senses:  Synset('even.s.02')
Definition:  equal in degree or extent or amount; or equally matched or balanced
Examples:  ['even amounts of butter and sugar', 'on even terms', 'it was a fifty-fifty (or even) split', 'had a fifty-fifty (or even) chance', 'an even fight']

High Frequency Senses:  Synset('even.a.03')
Definition:  being level or strai

The general idea is to look at what the POS are for the words surrounding them.
Adverbs will be followed by an adjective, verb or another adverb
Nouns are going to be followed by a verb or adjective, or preceded by a determiner or preposition
Verbs have to either follow or proceed a noun
Adjectives have to proceed a noun

# Question 2

In [34]:
from stat_parser import Parser
parser = Parser()
print(parser.parse("Book the cooks who cook the books."))
print()
# print(parser.parse("Though in many natural objects, whiteness refiningly enhances beauty, \
# as if imparting some special virtue of its own, as in marbles, \
# japonicas, and pearls; and though various nations have in some way \
# recognised a certain royal preeminence in this hue; even the \
# barbaric, grand old kings of Pegu placing the title Lord of the \
# White Elephants above all their other magniloquent ascriptions of dominion; \
# and the modern kings of Siam unfurling the same snow-white \
# quadruped in the royal standard; and the Hanoverian flag bearing the \
# one figure of a snow-white charger; and the great Austrian Empire, \
# Caesarian, heir to overlording Rome, having for the imperial colour \
# the same imperial hue; and though this pre-eminence in it applies to \
# the human race itself, giving the white man ideal mastership over \
# every dusky tribe; and though, besides, all this, whiteness has been \
# even made significant of gladness, for among the Romans a white stone \
# marked a joyful day; and though in other mortal sympathies and \
# symbolizings, this same hue is made the emblem of many touching, \
# noble things--the innocence of brides, the benignity of age; though \
# among the Red Men of America the giving of the white belt of wampum \
# was the deepest pledge of honour; though in many climes, whiteness \
# typifies the majesty of Justice in the ermine of the Judge, and \
# contributes to the daily state of kings and queens drawn by \
# milk-white steeds; though even in the higher mysteries of the most \
# august religions it has been made the symbol of the divine \
# spotlessness and power; by the Persian fire worshippers, the white \
# forked flame being held the holiest on the altar; and in the Greek \
# mythologies, Great Jove himself being made incarnate in a snow-white \
# bull; and though to the noble Iroquois, the midwinter sacrifice of \
# the sacred White Dog was by far the holiest festival of their \
# theology, that spotless, faithful creature being held the purest \
# envoy they could send to the Great Spirit with the annual tidings of \
# their own fidelity; and though directly from the Latin word for \
# white, all Christian priests derive the name of one part of their \
# sacred vesture, the alb or tunic, worn beneath the cassock; and \
# though among the holy pomps of the Romish faith, white is specially \
# employed in the celebration of the Passion of our Lord; though in the \
# Vision of St. John, white robes are given to the redeemed, and the \
# four-and-twenty elders stand clothed in white before the great-white \
# throne, and the Holy One that sitteth there white like wool; yet for \
# all these accumulated associations, with whatever is sweet, and \
# honourable, and sublime, there yet lurks an elusive something in the \
# innermost idea of this hue, which strikes more of panic to the soul \
# than that redness which affrights in blood."))

(S+NP
  (NN book)
  (NP (DT the) (NN cooks))
  (SBAR
    (WHNP (WP who))
    (S (VP (VB cook) (NP (DT the) (NNS books))) (. .))))



The above parser never completed for longest sentence this is probably due to the fact the algorithm's time complexity is O(n^3*|G|) where n is the length of the sentence and G is the length of the grammar.
Comparing "Book the cooks who cook the books." we see that the same answer was gotten. This parser goes further in creating a tree where we can see the sentence structure.

# Question 3

In [44]:
import sys
import getopt
import os
from collections import defaultdict

def segmentWords(s):
	return s.split()

def getGrammar(fileName):
	#code that takes the grammar from a file and stores it in a default dictionary, grammar_rules.
	f = open(fileName)
	grammar_rules = defaultdict(float)
	for line in f.read().splitlines():
		line = line.split()
		line_list =[]
		for word in line:
			line_list.append(word)
		if len(line_list)==3:
			grammar_rules[(line_list[0],line_list[1])] = float(line_list[2])
		if len(line_list) ==4:
			grammar_rules[(line_list[0],line_list[1], line_list[2])] = float(line_list[3])
	return grammar_rules

def getNonTerms(grammar_rules):
	"""
     * code that takes the grammar from the default dict , 
     * stores the nonterms in a set
    """
	x = []
	for rule in grammar_rules:
		x.append(rule[0])
	non_terms = set(x)
	return non_terms


def getSentenceList(fileName):
	f = open(fileName)
	sent_list = []
	for line in f.read().splitlines():
		sent_list.append(line)
	return sent_list


def parser(grammar, sentence, non_terms):
	#cky parser implementation
	sent = sentence.split()
	num_words = len(sent)
	score = [[defaultdict(float) for i in range(0,num_words+1)] for j in range(0,num_words+1)]
	back = [[defaultdict(float) for i in range(0,num_words+1)] for j in range(0,num_words+1)]

	for i in range(0,num_words):
		for A in non_terms:
			if (A,sent[i]) in grammar:				
				score[i][i+1][A] = grammar[(A,sent[i])]
			else:
				score[i][i+1][A] = 0.0
		added = True
		while added:
			added = False
			for A in non_terms:
				for B in non_terms:
					if (score[i][i+1][B]) > 0 and (A,B) in grammar:
						prob = grammar[(A,B)]*(score[i][i+1][B])
						if prob > (score[i][i+1][A]):
							score[i][i+1][A] = prob
							back[i][i+1][A] = B
							added = True

	for span in range(2,num_words+1):
		for begin in range(0,num_words+1 - span):
			end = begin + span
			for split in range(begin+1, end):
				for A in non_terms:
					for B in non_terms:
						for C in non_terms:
							prob=score[begin][split][B]*score[split][end][C]*grammar[(A,B,C)] 
							if prob > score[begin][end][A]:
								score[begin][end][A] = prob
								back[begin][end][A] = (split, B, C)
								

			added = True
			while added:
				added = False
				for A in non_terms:
					for B in non_terms:
						prob=grammar[(A,B)]*score[begin][end][B]
						if prob > score[begin][end][A]:
							score[begin][end][A] = prob
							back[begin][end][A]=B
							added = True

	print_pcfg(sent, score,back)
		


def print_pcfg(sent,score,back):
	"""
     * method to print the output of the cky parser
    """
	num_words=len(sent)
	str1=""
	for word in sent:
		str1+=" " + word

	print("PROCESSING SENTENCE:", str1)
	for span in range(1, num_words + 1):
		for begin in range(num_words - span + 1):
			end = begin + span
			str1=""
			for k in range(begin,end): str1 = str1 +" " + sent[k]
			print("\nSPAN:", str1)
			for key in score[begin][end]:
				if float(score[begin][end][key]) > 0.0:

					if key in back[begin][end]:
						print ("P(%s) = %.10f (BackPointer = %s)" % (key, score[begin][end][key], back[begin][end][key]))
					else:
						if span ==1:
							print ("P(%s%s) = %.10f" % (key, str1, score[begin][end][key]))
						else:
							print ("P(%s) = %.10f" %(key, score[begin][end][key]))

# def main():
#   (options, args) = getopt.getopt(sys.argv[1:], '')
#   grammar = defaultdict(float)
#   if len(args) == 2:
#     grammar = getGrammar(args[0])
#     non_terms = getNonTerms(grammar)
#     sentence_list = getSentenceList(args[1])
#     for sent in sentence_list:
#     	parser(grammar, sent, non_terms)
#   else:
#     print('Wrong input')
grammar = defaultdict(float)
grammar = getGrammar("/Users/tbennett/Documents/School/CS 557/CKY-parser-master 2/grammar_rules.txt")
non_terms = getNonTerms(grammar)
sentence_list = getSentenceList("/Users/tbennett/Documents/School/CS 557/CKY-parser-master 2/sents.txt")
for sent in sentence_list:
    parser(grammar, sent, non_terms)

PROCESSING SENTENCE:  book the cooks who cook the books

SPAN:  book
P(S) = 0.0010000000 (BackPointer = VP)
P(VP) = 0.0100000000 (BackPointer = V)
P(V book) = 0.1000000000

SPAN:  the
P(Det the) = 1.0000000000

SPAN:  cooks
P(Nominal) = 0.3500000000 (BackPointer = N)
P(N cooks) = 0.5000000000
P(NP) = 0.2000000000 (BackPointer = N)

SPAN:  who
P(Pronoun who) = 1.0000000000
P(NP) = 0.3000000000 (BackPointer = Pronoun)

SPAN:  cook
P(S) = 0.0060000000 (BackPointer = VP)
P(VP) = 0.0600000000 (BackPointer = V)
P(V cook) = 0.6000000000

SPAN:  the
P(Det the) = 1.0000000000

SPAN:  books
P(Nominal) = 0.1400000000 (BackPointer = N)
P(N books) = 0.2000000000
P(NP) = 0.0800000000 (BackPointer = N)

SPAN:  book the
P(S) = 0.0010000000 (BackPointer = VP)
P(VP) = 0.0100000000 (BackPointer = (1, 'V', 'Det'))

SPAN:  the cooks
P(NP) = 0.0700000000 (BackPointer = (2, 'Det', 'Nominal'))

SPAN:  cooks who
P(Nominal) = 0.0315000000 (BackPointer = (3, 'Nominal', 'NP'))
P(@VP_V) = 0.0600000000 (BackPointer

The code is a slight edit of https://github.com/janvi-palan/CKY-parser since I could not get the given one to work. The results are similar to the previous question, this program just shows all the steps. We also get every possible tag since we get to see each step. This allows us to see how the program built up to the final answer. Again the program could not run the longest sentence even after leaving it for 1.5 hours.